In [1]:
"""
Train on days 8 & 9
--
subm 023
predictors ['nextClick', 'nextClick_shift', 'app', 'device', 'os', 'channel', 'hour', 'ip_tcount', 'ip_tchan_count', 
'ip_app_count', 'ip_app_os_count', 'ip_app_os_var', 'ip_app_channel_var_day', 'ip_app_channel_mean_hour', 
'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'iadoc_mean_nc', 'ic_mean_nc']
train size:  104365809
valid size:  11596202
test size :  18790469

Training until validation scores don't improve for 30 rounds.
[10]	train's auc: 0.976291	val's auc: 0.975958
[20]	train's auc: 0.97987	val's auc: 0.979523
[30]	train's auc: 0.982051	val's auc: 0.981608
[40]	train's auc: 0.983125	val's auc: 0.982581
[50]	train's auc: 0.983691	val's auc: 0.983042
[60]	train's auc: 0.984024	val's auc: 0.983258
[70]	train's auc: 0.984403	val's auc: 0.983474
[80]	train's auc: 0.984647	val's auc: 0.983642
[90]	train's auc: 0.984878	val's auc: 0.983743
[100]	train's auc: 0.985067	val's auc: 0.983787
[110]	train's auc: 0.985254	val's auc: 0.983922
[120]	train's auc: 0.985415	val's auc: 0.984008
[130]	train's auc: 0.985534	val's auc: 0.984037
[140]	train's auc: 0.985662	val's auc: 0.984096
[150]	train's auc: 0.985788	val's auc: 0.984134
[160]	train's auc: 0.985895	val's auc: 0.984157
[170]	train's auc: 0.985993	val's auc: 0.984149
[180]	train's auc: 0.986095	val's auc: 0.984178
[190]	train's auc: 0.986173	val's auc: 0.984187
[200]	train's auc: 0.986252	val's auc: 0.984202
[210]	train's auc: 0.986323	val's auc: 0.984217
[220]	train's auc: 0.986392	val's auc: 0.984249
[230]	train's auc: 0.986466	val's auc: 0.984234
[240]	train's auc: 0.986546	val's auc: 0.984255
[250]	train's auc: 0.986608	val's auc: 0.984257
[260]	train's auc: 0.986667	val's auc: 0.984251
[270]	train's auc: 0.98673	val's auc: 0.984253
[280]	train's auc: 0.986789	val's auc: 0.984256
Early stopping, best iteration is:
[253]	train's auc: 0.986624	val's auc: 0.984261

Model Report
bst1.best_iteration:  253
[5459.385001420975]: model training time

"""

import os
import gc
import datetime
import time

import pandas as pd
import numpy as np
import lightgbm as lgb
import feather

import matplotlib.pyplot as plt
import seaborn as sns

inpath = '../input/'
suffix = ''
outpath = ''
savepath = ''
debug=0


In [2]:
nrows=      184903891-1
#train_size=  75000000
#val_size=     2500000
#offset=      75000000
#frm=nrows-offset
#to=frm+train_size

#day 8 starting from=68941878
#day 9 starts 131886953
#day 9 ends 184903889

#day 9 hour 4 starts; 144708152
#day 9 hour 4 ends; 148740842

train_size=  62945075
val_size=    53016936
offset=      68941878
frm=offset
to=frm+train_size+val_size


In [3]:
frm, to

(68941878, 184903889)

In [4]:


def do_count( df, group_cols, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Aggregating by ", group_cols , '...' )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_countuniq( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Counting unqiue ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 
    
def do_cumcount( df, group_cols, counted, agg_name, agg_type='uint32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Cumulative count by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount()
    df[agg_name]=gp.values
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_mean( df, group_cols, counted, agg_name, agg_type='float32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Calculating mean of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].mean().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 

def do_var( df, group_cols, counted, agg_name, agg_type='float32', show_max=False, show_agg=True ):
    if show_agg:
        print( "Calculating variance of ", counted, " by ", group_cols , '...' )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].var().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return df 


def DO(frm, to):
    dtypes = {
            'ip'            : 'uint32',
            'app'           : 'uint16',
            'device'        : 'uint16',
            'os'            : 'uint16',
            'channel'       : 'uint16',
            'is_attributed' : 'uint8',
            'click_id'      : 'uint32',
            }
    
    print('loading train data...')
    train_df = feather.read_dataframe('/kaggle1/td-cache/train_base.feather')
    train_df = train_df.iloc[frm:to]
        
    gc.collect()
    
    print('loading test data...')
    test_df = feather.read_dataframe('/kaggle1/td-cache/test_base.feather')
    train_df=train_df.append(test_df)
    
    del test_df
    gc.collect()
    
    print('Extracting new features...')
    
    gc.collect()
    train_df = do_countuniq( train_df, ['ip'], 'channel', 'X0', 'uint8', show_max=True ); gc.collect()
    train_df = do_cumcount( train_df, ['ip', 'device', 'os'], 'app', 'X1', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['ip', 'day'], 'hour', 'X2', 'uint8', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['ip'], 'app', 'X3', 'uint8', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['ip', 'app'], 'os', 'X4', 'uint8', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['ip'], 'device', 'X5', 'uint16', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['app'], 'channel', 'X6', show_max=True ); gc.collect()
    train_df = do_cumcount( train_df, ['ip'], 'os', 'X7', show_max=True ); gc.collect()
    train_df = do_countuniq( train_df, ['ip', 'device', 'os'], 'app', 'X8', show_max=True ); gc.collect()
    train_df = do_count( train_df, ['ip', 'day', 'hour'], 'ip_tcount', show_max=True ); gc.collect()
    train_df = do_count( train_df, ['ip', 'app'], 'ip_app_count', show_max=True ); gc.collect()
    train_df = do_count( train_df, ['ip', 'app', 'os'], 'ip_app_os_count', 'uint16', show_max=True ); gc.collect()    
    train_df = do_var( train_df, ['ip', 'day', 'channel'], 'hour', 'ip_tchan_count', show_max=True ); gc.collect()
    train_df = do_var( train_df, ['ip', 'app', 'os'], 'hour', 'ip_app_os_var', show_max=True ); gc.collect()
    train_df = do_var( train_df, ['ip', 'app', 'channel'], 'day', 'ip_app_channel_var_day', show_max=True ); gc.collect()
    train_df = do_mean( train_df, ['ip', 'app', 'channel'], 'hour', 'ip_app_channel_mean_hour', show_max=True ); gc.collect()

    print('doing nextClick')
    #predictors=[]
    
    new_feature = 'nextClick'
    filename='nextClick_%d_%d.csv'%(frm,to)

    if os.path.exists(filename):
        print('loading from save file')
        QQ=pd.read_csv(filename, header=None)[0].values
    else:
        D=2**26
        train_df['category'] = (train_df['ip'].astype(str) + "_" + train_df['app'].astype(str) + "_" + train_df['device'].astype(str) \
            + "_" + train_df['os'].astype(str)).apply(hash) % D
        click_buffer= np.full(D, 3000000000, dtype=np.uint32)

        train_df['epochtime']= train_df['click_time'].astype(np.int64) // 10 ** 9
        next_clicks= []
        for category, t in zip(reversed(train_df['category'].values), reversed(train_df['epochtime'].values)):
            next_clicks.append(click_buffer[category]-t)
            click_buffer[category]= t
        del(click_buffer)
        QQ= list(reversed(next_clicks))

        if not debug:
            print('saving')
            pd.DataFrame(QQ).to_csv(filename,index=False)
            
    to_drop = [x for x in train_df.columns if x in ['epochtime','category','click_time']]
    train_df.drop(to_drop, axis=1, inplace=True)

    train_df[new_feature] = pd.Series(QQ).astype('float32')
    #predictors.append(new_feature)

    train_df[new_feature+'_shift'] = train_df[new_feature].shift(+1).values
    #predictors.append(new_feature+'_shift')
    
    del QQ
    gc.collect()

    train_df['ip_tcount'] = train_df['ip_tcount'].astype('uint16')
    train_df['ip_app_count'] = train_df['ip_app_count'].astype('uint16')
    train_df['ip_app_os_count'] = train_df['ip_app_os_count'].astype('uint16')

    print("vars and data type: ")
    train_df.info()
    
    train_df.reset_index(drop=True, inplace=True)
    feather.write_dataframe(train_df, '9671_train_from{}_to{}.feather'.format(frm, to))

    return train_df
    



In [5]:
import feather
import datetime

print(datetime.datetime.now())
fname = '9671_train_from{}_to{}.feather'.format(frm, to)
if not os.path.exists(fname):
    print('generating')
    DO(frm, to)
train_df = feather.read_dataframe(fname)    
print(datetime.datetime.now())

2018-04-20 16:16:57.752146
2018-04-20 16:18:28.357997


In [6]:
#import feather
#df = feather.read_dataframe('/kaggle1/td-cache/train_base.feather')
    

In [7]:
#train_df.nextClick.max()

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134752480 entries, 0 to 134752479
Data columns (total 26 columns):
app                         uint16
channel                     uint16
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               uint8
os                          uint16
X0                          uint8
X1                          uint32
X2                          uint8
X3                          uint8
X4                          uint8
X5                          uint16
X6                          uint32
X7                          uint32
X8                          uint32
ip_tcount                   uint16
ip_app_count                uint16
ip_app_os_count             uint16
ip_tchan_count              float32
ip_app_os_var               float32
ip_app_channel_var_day      float32
ip_app_channel_mean_hour    float32
nextClick                   float32
nextClick_

In [9]:
def lgb_modelfit_nocv(params, train_df, val_dfs, predictors, target='target', objective='binary', metrics='auc',
                 feval=None, early_stopping_rounds=20, num_boost_round=3000, verbose_eval=10, categorical_features=None):
    
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': objective,
        'metric':metrics,
        'nthread': 8,
        'verbose': 0,
        'metric':metrics
    }

    lgb_params.update(params)

    print("preparing validation datasets")

    dtrain = lgb.Dataset(train_df[predictors].values, label=train_df[target].values,
                         feature_name=predictors,
                         categorical_feature=categorical_features)
    valid_names = ['train']
    valid_sets = [dtrain]
    for val_name, val_df in val_dfs.items():
        valid_names.append(val_name)
        dvalid = lgb.Dataset(val_df[predictors].values, 
                             label=val_df[target].values,
                             feature_name=predictors,
                             categorical_feature=categorical_features)
        valid_sets.append(dvalid)

    evals_results = {}

    bst1 = lgb.train(lgb_params, 
                     dtrain, 
                     valid_sets=valid_sets,
                     valid_names=valid_names, 
                     evals_result=evals_results, 
                     num_boost_round=num_boost_round,
                     early_stopping_rounds=early_stopping_rounds,
                     verbose_eval=10, 
                     feval=feval)

    print("\nModel Report")
    print("bst1.best_iteration: ", bst1.best_iteration)
    #print(metrics+":", evals_results['valid'][metrics][bst1.best_iteration-1])

    return bst1, bst1.best_iteration, evals_results


In [10]:
categorical = ['app', 'device', 'os', 'channel', 'hour']

predictors = []
predictors.extend(['nextClick', 'nextClick_shift'])
predictors.extend([
    'app','device','os', 'channel', 'hour', #'day', 
    'ip_tcount', 'ip_tchan_count', 'ip_app_count',
    'ip_app_os_count', 
    'ip_app_os_var',
    'ip_app_channel_var_day','ip_app_channel_mean_hour',
    'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'])

print('categorical:', categorical)

categorical: ['app', 'device', 'os', 'channel', 'hour']


In [11]:
def _add_mean_nc(df, group_cols, name):
    gp = df[group_cols + ['nextClick']].groupby(by=group_cols).nextClick.mean().rename(columns={'nextClick': name}).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    gc.collect()
    return df

train_df = _add_mean_nc(train_df, ['ip', 'app', 'device', 'os', 'channel'], 'iadoc_mean_nc')
train_df = _add_mean_nc(train_df, ['ip', 'channel'], 'ic_mean_nc')
predictors.extend(['iadoc_mean_nc', 'ic_mean_nc'])

In [11]:
#train_df = do_count( train_df, ['ip', 'app', 'channel'], 'ip_app_channel_count', 'uint16', show_max=True ); gc.collect()
#train_df = do_count( train_df, ['ip', 'app', 'os', 'hour'], 'ip_app_os_hour_count', 'uint16', show_max=True ); gc.collect()
#predictors.extend(['ip_app_channel_count', 'ip_app_os_hour_count'])


In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 134752480 entries, 0 to 134752479
Data columns (total 28 columns):
app                         uint16
channel                     uint16
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               uint8
os                          uint16
X0                          uint8
X1                          uint32
X2                          uint8
X3                          uint8
X4                          uint8
X5                          uint16
X6                          uint32
X7                          uint32
X8                          uint32
ip_tcount                   uint16
ip_app_count                uint16
ip_app_os_count             uint16
ip_tchan_count              float32
ip_app_os_var               float32
ip_app_channel_var_day      float32
ip_app_channel_mean_hour    float32
nextClick                   float32
nextClick_

In [14]:
#train_df['iadoc_mean_nc'] = train_df['0_x']
#train_df['ic_mean_nc'] = train_df['0_y']
del train_df['0_x']
del train_df['0_y']


<class 'pandas.core.frame.DataFrame'>
Int64Index: 134752480 entries, 0 to 134752479
Data columns (total 30 columns):
app                         uint16
channel                     uint16
day                         uint8
device                      uint16
hour                        uint8
ip                          uint32
is_attributed               uint8
os                          uint16
X0                          uint8
X1                          uint32
X2                          uint8
X3                          uint8
X4                          uint8
X5                          uint16
X6                          uint32
X7                          uint32
X8                          uint32
ip_tcount                   uint16
ip_app_count                uint16
ip_app_os_count             uint16
ip_tchan_count              float32
ip_app_os_var               float32
ip_app_channel_var_day      float32
ip_app_channel_mean_hour    float32
nextClick                   float32
nextClick_

In [16]:
from sklearn.model_selection import train_test_split

def DO2(train_df, predictors, categorical):

    target = 'is_attributed'
    print('predictors', predictors)

    frm, to = (68941878, 184903889)
    
    len_train = to - frm
    
    test_df = train_df[len_train:]
    
    train_df, val_df = train_test_split(train_df[:len_train], test_size=.1)

    print("train size: ", len(train_df))
    print("valid size: ", len(val_df))
    print("test size : ", len(test_df))
    
    val_dfs = {
        'val':  val_df,
    }    
    
    gc.collect()

    print("Training...")
    start_time = time.time()

    params = {
        'learning_rate': 0.2,
        #'is_unbalance': 'true', # replaced with scale_pos_weight argument
        'num_leaves': 15,  # 2^max_depth - 1
        'max_depth': 4,  # -1 means no limit
        'min_child_samples': 100,  # Minimum number of data need in a child(min_data_in_leaf)
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.7,  # Subsample ratio of the training instance.
        'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
        'colsample_bytree': 0.9,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'scale_pos_weight':200, # because training data is extremely unbalanced 
        'subsample_for_bin': 200000,  # Number of samples for constructing bin
        'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 0,  # L1 regularization term on weights
        'reg_lambda': 0,  # L2 regularization term on weights
        
    }
    bst, best_iteration, evals_results = lgb_modelfit_nocv(params, 
                            train_df, 
                            val_dfs, 
                            predictors, 
                            target, 
                            objective='binary', 
                            metrics='auc',
                            early_stopping_rounds=30, 
                            verbose_eval=True, 
                            num_boost_round=1000, 
                            categorical_features=categorical)

    print('[{}]: model training time'.format(time.time() - start_time))
    del train_df
    del val_df
    gc.collect()

    
    #print('Plot feature importances...')
    #ax = lgb.plot_importance(bst, max_num_features=100)
    #plt.show()
    

    return bst, evals_results
    

In [17]:
#day 9 starts 131886953
#day 9 ends 184903889
bst, evals_results = DO2(train_df, predictors, categorical)

predictors ['nextClick', 'nextClick_shift', 'app', 'device', 'os', 'channel', 'hour', 'ip_tcount', 'ip_tchan_count', 'ip_app_count', 'ip_app_os_count', 'ip_app_os_var', 'ip_app_channel_var_day', 'ip_app_channel_mean_hour', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'iadoc_mean_nc', 'ic_mean_nc']
train size:  104365809
valid size:  11596202
test size :  18790469
Training...
preparing validation datasets


/opt/miniconda2/envs/talkingdata/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/opt/miniconda2/envs/talkingdata/lib/python3.6/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 30 rounds.
[10]	train's auc: 0.976291	val's auc: 0.975958
[20]	train's auc: 0.97987	val's auc: 0.979523
[30]	train's auc: 0.982051	val's auc: 0.981608
[40]	train's auc: 0.983125	val's auc: 0.982581
[50]	train's auc: 0.983691	val's auc: 0.983042
[60]	train's auc: 0.984024	val's auc: 0.983258
[70]	train's auc: 0.984403	val's auc: 0.983474
[80]	train's auc: 0.984647	val's auc: 0.983642
[90]	train's auc: 0.984878	val's auc: 0.983743
[100]	train's auc: 0.985067	val's auc: 0.983787
[110]	train's auc: 0.985254	val's auc: 0.983922
[120]	train's auc: 0.985415	val's auc: 0.984008
[130]	train's auc: 0.985534	val's auc: 0.984037
[140]	train's auc: 0.985662	val's auc: 0.984096
[150]	train's auc: 0.985788	val's auc: 0.984134
[160]	train's auc: 0.985895	val's auc: 0.984157
[170]	train's auc: 0.985993	val's auc: 0.984149
[180]	train's auc: 0.986095	val's auc: 0.984178
[190]	train's auc: 0.986173	val's auc: 0.984187
[200]	train's auc: 0.986252	val's au

In [18]:
def predict(train_df, frm, to, predictors, bst, best_iteration=-1):
    len_train = to - frm    
    test_df = train_df[len_train:]
    
    print("Predicting...")
    click_ids = pd.read_csv('../input/test.csv', usecols=['click_id'])['click_id'].values
    
    sub = pd.DataFrame()    
    sub['click_id'] = click_ids
        
    gc.collect()
        
    sub['is_attributed'] = bst.predict(test_df[predictors],num_iteration=best_iteration)
                            
    print("writing...")
    sub.to_csv('subm_000_{}.csv'.format(best_iteration), index=False, float_format='%.9f')
    print("done...")    

In [19]:
predict(train_df, frm, to, predictors, bst, 150) #subm22

Predicting...
writing...
done...


In [20]:
predict(train_df, frm, to, predictors, bst, 250) #subm23

Predicting...
writing...
done...
